In [1]:
import re
import csv
import nltk
import keras
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_path = '/kaggle/input/4-classes-sinhala-original-dataset/4 Classes (Original Dataset).csv'
stop_words = '/kaggle/input/stop-words/stop words.txt'
stem_dictionary = '/kaggle/input/stem-dictionary/stem_dictionary.txt'

In [3]:
df = pd.read_csv(df_path)

In [4]:
df.head()

,Class Index,Paragraph
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...


## Cleaning paragraph (Tokenizing, Removing punctuations, Removing non-Latin characters)

In [5]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [6]:
df['Cleaned Paragraph'] = df['Paragraph'].apply(lambda x: clean_text(x))
df.head()

,Class Index,Paragraph,Cleaned Paragraph
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...,"[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට..."
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...,"[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


## Removing Stopwords

In [7]:
stopwordsreader = open(stop_words, 'r', encoding='utf-8')
reader = csv.reader(stopwordsreader)

swords = []
for row in reader:
    swords.append(row[0])
print(swords)

['සහ', 'සමග', 'සමඟ', 'අහා', 'ආහ්', 'ආ', 'ඕහෝ', 'අනේ', 'අඳෝ', 'අපොයි', 'අපෝ', 'අයියෝ', 'ආයි', 'ඌයි', 'චී', 'චිහ්', 'චික්', 'හෝ\u200d', 'දෝ', 'දෝහෝ', 'මෙන්', 'සේ', 'වැනි', 'බඳු', 'වන්', 'අයුරු', 'අයුරින්', 'ලෙස', 'වැඩි', 'ශ්\u200dරී', 'හා', 'ය', 'නිසා', 'නිසාවෙන්', 'බවට', 'බව', 'බවෙන්', 'නම්', 'වැඩි', 'සිට', 'දී', 'මහා', 'මහ', 'පමණ', 'පමණින්', 'පමන', 'වන', 'විට', 'විටින්', 'මේ', 'මෙලෙස', 'මෙයින්', 'ඇති', 'ලෙස', 'සිදු', 'වශයෙන්', 'යන', 'සඳහා', 'මගින්', 'හෝ\u200d', 'ඉතා', 'ඒ', 'එම', 'ද', 'අතර', 'විසින්', 'සමග', 'පිළිබඳව', 'පිළිබඳ', 'තුළ', 'බව', 'වැනි', 'මහ', 'මෙම', 'මෙහි', 'මේ', 'වෙත', 'වෙතින්', 'වෙතට', 'වෙනුවෙන්', 'වෙනුවට', 'වෙන', 'ගැන', 'නෑ', 'අනුව', 'නව', 'පිළිබඳ', 'විශේෂ', 'දැනට', 'එහෙන්', 'මෙහෙන්', 'එහේ', 'මෙහේ', 'ම', 'තවත්', 'තව ', 'සහ', 'දක්වා', 'ට', 'ගේ', 'එ', 'ක', 'ක්', 'බවත්', 'බවද', 'මත', 'ඇතුලු', 'ඇතුළු', 'මෙසේ', 'වඩා', 'වඩාත්ම', 'නිති', 'නිතිත්', 'නිතොර', 'නිතර', 'ඉක්බිති', 'දැන්', 'යලි', 'පුන', 'ඉතින්', 'සිට', 'සිටන්', 'පටන්', 'තෙක්', 'දක්වා', 'සා', 'තාක්', 'තුවක්', 'පවා', 'ද

In [8]:
def remove_stop_words(tokens):
    regex = re.compile(u'[^\u0D80-\u0DFF]', re.UNICODE)
    tokens = [regex.sub('', w) for w in tokens]
    tokens = [c for c in tokens if c not in string.punctuation]
    return tokens

In [9]:
df['Stopwords Removed (Paragraph)'] = df['Cleaned Paragraph'].apply(lambda x: remove_stop_words(x))
df.head()

,Class Index,Paragraph,Cleaned Paragraph,Stopwords Removed (Paragraph)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...,"[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,..."
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට..."
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...,"[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ...","[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ..."
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,..."
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක..."


## Stemming

In [10]:
stem_dict = open(stem_dictionary, "r", encoding='utf-8')
stem_dict = map(lambda s: s.strip(), stem_dict)

stem_dictionary = {}

for s in stem_dict:
    s = s.split("\t")
    stem_dictionary[s[0]] = s[1]

In [11]:
def stemming_words(tokens):
    for k, v in enumerate(tokens):
        tokens[k] = stem_dictionary.get(v, v)
    tokens = " ".join([c for c in tokens if c not in string.punctuation])
    return tokens

In [12]:
df['Stemming Words (Paragraph)'] = df['Stopwords Removed (Paragraph)'].apply(lambda x: stemming_words(x))
df.head()

,Class Index,Paragraph,Cleaned Paragraph,Stopwords Removed (Paragraph),Stemming Words (Paragraph)
0,3,වෝල් වීදිය නැවතත් ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය...,"[වෝල්, වීදිය, නැවතත්, ධනාත්මක, ආර්ථිකයක්, ලබා,...","[වෝල්, වීදිය, නැවත, ධනාත්මක, ආර්ථිකයක්, ලබා, ග...",වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්‍යවකාශය දෙස බලයි (රොයිටර්) ...,"[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට...","[කාර්ලයිල්, වාණිජ, අභ්යවකාශය, දෙස, බලයි, රොයිට...",කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් - බොරතෙල් මිල ඉහළ යාම ...,"[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ...","[තෙල්, සහ, ආර්ථිකය, රොයිටර්, බොරතෙල්, මිල, ඉහළ...",තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාකය ප්‍රධාන දකුණු නල මාර්ගයෙන් (රොයිටර්) තෙල...,"[ඉරාකය, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්,...","[ඉරාක, ප්රධාන, දකුණු, නල, මාර්ගයෙන්, රොයිටර්, ...",ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත් ජනපද ආර්ථිකයට (AFP) නව තර්ජනයක් එල්ල කර...,"[එක්සත්, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, ක...","[එක්සත, ජනපද, ආර්ථිකයට, නව, තර්ජනයක්, එල්ල, කර...",එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [13]:
df.drop(df.columns[[1, 2, 3]], axis=1, inplace=True)
df.head()

,Class Index,Stemming Words (Paragraph)
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [14]:
df = df.rename(columns={'Class Index': 'Labels', 'Stemming Words (Paragraph)': 'Paragraph'})

In [15]:
df.head()

,Labels,Paragraph
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


## Saving new preprocessed dataframe to CSV

In [16]:
df.to_csv('4 Classes (Preprocessed Dataset).csv', index=False)

## Splitting dataset to train and test

In [17]:
df = pd.read_csv('/kaggle/working/4 Classes (Preprocessed Dataset).csv')

In [18]:
df.head()

,Labels,Paragraph
0,3,වෝල් වීදිය නැවත ධනාත්මක ආර්ථිකයක් ලබා ගත්තේය න...
1,3,කාර්ලයිල් වාණිජ අභ්යවකාශය දෙස බලයි රොයිටර් රොය...
2,3,තෙල් සහ ආර්ථිකය රොයිටර් බොරතෙල් මිල ඉහළ යාම සහ...
3,3,ඉරාක ප්රධාන දකුණු නල මාර්ගයෙන් රොයිටර් තෙල් අප...
4,3,එක්සත ජනපද ආර්ථිකයට නව තර්ජනයක් එල්ල කරමින් තෙ...


In [19]:
# Split the data into training and testing sets (e.g., 80% train, 20% test)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
# Save the training and testing sets as separate CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)